Reference: https://github.com/alineberry/my-movie-recommender/blob/master/notebooks/movie_similarity/autoencoder.ipynb

# Models: Movie Overview Sparse Autoencoder

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../lib')

import numpy as np
import pandas as pd
from bunch import Bunch

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam

import pytorch_common.util as pu
from pytorch_common.modules.fn import Fn
from pytorch_common.callbacks import SaveBestModel
from pytorch_common.callbacks.output import Logger

from pytorch_common.util import set_device_name, \
                                get_device, \
                                LoggerBuilder

import model as ml
import data as dt
import data.dataset as ds

import data.plot as pl
import data as dtjo

import logging
import random

import recommender as rc

<Figure size 432x288 with 0 Axes>

## Setup

In [3]:
pu.LoggerBuilder().on_console().build()

<RootLogger root (INFO)>

In [4]:
pu.set_device_name('gpu')

In [5]:
pu.get_device()

device(type='cuda', index=0)

In [6]:
cpu = torch.device("cpu")
gpu = pu.get_device()

In [7]:
torch.cuda.is_available()

True

In [8]:
torch.__version__

'1.11.0'

In [9]:
def set_seed(value):
    random.seed(value)
    np.random.seed(value)
    torch.manual_seed(value)

In [10]:
set_seed(42)

In [11]:
FIELD = 'overview'
WEIGHTS_PATH   = f'../weights/{FIELD}-tf-idf-sparse-auto-encoder.pt'
EMBEDDING_PATH = f'../datasets/movie_{FIELD}_embedding.json'

## Carga de dataset

In [12]:
def to_tensor(obs, device, columns): 
    data = obs[columns]
    if type(data) == pd.DataFrame:
        data = data.values
    return torch.tensor(data).to(device)

transform_fn = lambda obs, device: to_tensor(obs, device, [f'movie_{FIELD}'])

dataset = ds.MovieLensTMDBDatasetFactory.from_path(
    transform        = transform_fn,
    target_transform = transform_fn,
    device           = cpu,
    filter_fn        = lambda df: df[(df['user_movie_rating_year'] >= 2005) & (df['user_movie_rating_year'] <= 2019)]
)
dataset.info

<class 'pandas.core.frame.DataFrame'>
Int64Index: 191540 entries, 0 to 191539
Data columns (total 15 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   user_id                      191540 non-null  int64         
 1   user_seq                     191540 non-null  int64         
 2   user_movie_tags              191540 non-null  object        
 3   user_movie_rating            191540 non-null  int64         
 4   user_movie_rating_timestamp  191540 non-null  datetime64[ns]
 5   user_movie_rating_year       191540 non-null  int64         
 6   movie_id                     191540 non-null  int64         
 7   movie_seq                    191540 non-null  int64         
 8   movie_title                  191540 non-null  string        
 9   movie_genres                 191540 non-null  object        
 10  movie_for_adults             191540 non-null  bool          
 11  movie_original_language   

Select movies overview and add new curated tokens column:

In [26]:
columns = ['movie_id', 'movie_release_year', 'movie_title', f'movie_{FIELD}']

movie_data = dataset \
    .data \
    .pipe(dt.select, columns) \
    .pipe(dt.distinct, ['movie_id']) \
    .pipe(dt.rename, {
        'movie_id': 'id', 
        'movie_title': 'title', 
        f'movie_{FIELD}': FIELD
    }) \
    .pipe(dt.tokenize, FIELD) \
    .pipe(dt.reset_index)

movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18515 entries, 0 to 18514
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  18515 non-null  int64 
 1   movie_release_year  18515 non-null  int64 
 2   title               18515 non-null  string
 3   overview            18515 non-null  string
 4   overview_tokens     18515 non-null  object
dtypes: int64(2), object(1), string(2)
memory usage: 723.4+ KB


In [27]:
movie_data

,id,movie_release_year,title,overview,overview_tokens
0,1,1995,Toy Story,"Led by Woody, Andy's toys live happily in his ...",led woody andy toys live happily room andy bir...
1,2355,1998,"Bug's Life, A","On behalf of ""oppressed bugs everywhere,"" an i...",behalf oppressed bugs inventive ant named flik...
2,3114,1999,Toy Story 2,"Andy heads off to Cowboy Camp, leaving his toy...",andy heads cowboy camp leaving toys devices th...
3,4306,2001,Shrek,It ain't easy bein' green -- especially if you...,ai easy bein green especially likable albeit s...
4,4886,2001,"Monsters, Inc.","James Sullivan and Mike Wazowski are monsters,...",james sullivan mike wazowski monsters earn liv...
...,...,...,...,...,...
18510,173173,2017,This Is Not What I Expected,"Lu Jin is a handsome, wealthy hotel executive ...",lu jin handsome wealthy hotel executive drive ...
18511,174399,2012,Daddy's Little Girl,After the police find Derek’s daughter brutall...,police find derek daughter brutally murdered b...
18512,174443,2016,American Wrestler: The Wizard,"In 1980, a teenage boy escapes the unrest in I...",teenage boy escapes unrest iran face hostility...
18513,174505,2016,Besetment,"After struggling to find employment, Amanda ta...",struggling find employment amanda takes hotel ...


In [28]:
tfidf = movie_data.pipe(dt.tf_idf, f'{FIELD}_tokens')

tfidf.shape

(18515, 23051)

## Definicion del modelo

In [29]:
def train(auto_encoder, tfidf, params):
    train_set = DataLoader(
        ds.TfIdfDataset(tfidf), 
        params.batch_size, 
        num_workers=params.n_workers, 
        pin_memory=True,
        shuffle=True
    )
    ml.AutoEncoderTrainer(auto_encoder).fit(
        train_set,
        loss_fn = ml.MSELossFn(reduction='elementwise_mean'),
        epochs  = params.epochs,
        encoder_optimizer = Adam(auto_encoder.encoder.parameters(), lr= params.lr),
        decoder_optimizer = Adam(auto_encoder.decoder.parameters(), lr= params.lr),
        callbacks=[Logger(['time', 'epoch', 'train_loss'])]
    )

## Entrenamiento

In [30]:
params = Bunch({
    'lr': 0.01,
    'epochs': 20,
    'n_workers': 24,
    'batch_size': 128,
    'sequence_size':  tfidf.shape[1],
    'intermediate_size': 5000,
    'encoded_size': 1000,
    'experiment_name': f'{FIELD}-tf-idf-sparse-auto-encoder',
    'device': get_device()
})

In [31]:
auto_encoder = ml.AutoEncoder(
    params.sequence_size, 
    params.intermediate_size, 
    params.encoded_size
).to(get_device())
print(auto_encoder)

AutoEncoder(
  (encoder): Encoder(
    (mlp): Sequential(
      (0): Linear(in_features=23051, out_features=5000, bias=True)
      (1): BatchNorm1d(5000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Dropout(p=0.2, inplace=False)
      (4): Linear(in_features=5000, out_features=1000, bias=True)
      (5): BatchNorm1d(1000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU(inplace=True)
      (7): Dropout(p=0.2, inplace=False)
    )
  )
  (decoder): Decoder(
    (mlp): Sequential(
      (0): Linear(in_features=1000, out_features=5000, bias=True)
      (1): BatchNorm1d(5000, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Dropout(p=0.2, inplace=False)
      (4): Linear(in_features=5000, out_features=23051, bias=True)
      (5): BatchNorm1d(23051, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): Sigmoid()
    )
  )
)

In [32]:
train(auto_encoder, tfidf, params)

2022-07-24 12:38:35,523 - INFO - {'time': '0:00:07.92', 'epoch': 1, 'train_loss': 0.13628351770598313}
2022-07-24 12:38:43,368 - INFO - {'time': '0:00:07.84', 'epoch': 2, 'train_loss': 0.04234038751957745}
2022-07-24 12:38:51,175 - INFO - {'time': '0:00:07.81', 'epoch': 3, 'train_loss': 0.0235675530700848}
2022-07-24 12:38:59,016 - INFO - {'time': '0:00:07.84', 'epoch': 4, 'train_loss': 0.014690781416821069}
2022-07-24 12:39:06,873 - INFO - {'time': '0:00:07.86', 'epoch': 5, 'train_loss': 0.006722408661554599}
2022-07-24 12:39:14,678 - INFO - {'time': '0:00:07.80', 'epoch': 6, 'train_loss': 0.003975038718560646}
2022-07-24 12:39:22,454 - INFO - {'time': '0:00:07.78', 'epoch': 7, 'train_loss': 0.0026282713058051366}
2022-07-24 12:39:30,199 - INFO - {'time': '0:00:07.74', 'epoch': 8, 'train_loss': 0.0019257104907441755}
2022-07-24 12:39:38,106 - INFO - {'time': '0:00:07.91', 'epoch': 9, 'train_loss': 0.00149213083425601}
2022-07-24 12:39:46,018 - INFO - {'time': '0:00:07.91', 'epoch': 10

In [33]:
torch.save(auto_encoder.state_dict(), WEIGHTS_PATH)

## Generacion de embeddings

In [34]:
embedding = auto_encoder.to(cpu).encode_from_batch(torch.tensor(tfidf.toarray()))
embedding.shape

torch.Size([18515, 1000])

In [35]:
movie_data = movie_data \
    .pipe(dt.append_emb_vectors, embedding, FIELD)

movie_data.to_json(EMBEDDING_PATH)
movie_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18515 entries, 0 to 18514
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  18515 non-null  int64 
 1   movie_release_year  18515 non-null  int64 
 2   title               18515 non-null  string
 3   overview            18515 non-null  string
 4   overview_tokens     18515 non-null  object
 5   overview_embedding  18515 non-null  object
dtypes: int64(2), object(2), string(2)
memory usage: 868.0+ KB


## Evaluación

In [36]:
df = pd.read_json(EMBEDDING_PATH)

In [37]:
recommender = rc.DistanceMatrixRecommender(
    df,
    column  = f'{FIELD}_embedding', 
    device  = get_device()
)

Building Distances Matrix:   0%|          | 0/18515 [00:00<?, ?it/s]

In [38]:
result = recommender.recommend(item_index=0)
result.show()


Recommender: overview
Item


,id,title
0,1,Toy Story


Recommendations


,index,distance,id,title,overview
0,0,0.000000,1,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,11989,0.026161,54341,Wild Tigers I Have Known,A lyrical telling of the coming of age of a 13...
2,19,0.027491,78499,Toy Story 3,"Woody, Buzz, and the rest of Andy's toys haven..."
3,10152,0.028743,5569,"Last House on the Left, The","Mari and Phyllis go to the ""big city"" to see t..."
4,1623,0.029026,5785,Jackass: The Movie,Johnny Knoxville and his crazy friends appear ...
